# Data Preparation and Cleaning 

In [1]:
#Import libraries
# Standard Packages
import pandas as pd
import numpy as np

# Suppress future and deprecation warnings
import warnings
warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#Display Full Dataframe width
pd.options.display.max_columns = None
pd.options.display.width = None

### House Sales Dataset

In [2]:
#Import house sales data
kc_family_house_df = pd.read_csv('/Users/Aidan/Documents/Flatiron/Phase_2/King-County-House-Sales-/data/kc_house_data.csv')

#Reset the index
kc_family_house_df.reset_index(drop=True,inplace= True)

#Preview first 5 rows of subset
kc_family_house_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long
0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,NO,NONE,Good,7 Average,Gas,PUBLIC,1180,0,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.19052
1,8910500230,12/13/2021,920000.0,5,2.5,2770,6703,1.0,NO,NO,YES,AVERAGE,Average,7 Average,Oil,PUBLIC,1570,1570,0,240,1950,0,"11231 Greenwood Avenue North, Seattle, Washing...",47.711525,-122.35591
2,1180000275,9/29/2021,311000.0,6,2.0,2880,6156,1.0,NO,NO,NO,AVERAGE,Average,7 Average,Gas,PUBLIC,1580,1580,0,0,1956,0,"8504 South 113th Street, Seattle, Washington 9...",47.502045,-122.22520
3,1604601802,12/14/2021,775000.0,3,3.0,2160,1400,2.0,NO,NO,NO,AVERAGE,Average,9 Better,Gas,PUBLIC,1090,1070,200,270,2010,0,"4079 Letitia Avenue South, Seattle, Washington...",47.566110,-122.29020
4,8562780790,8/24/2021,592500.0,2,2.0,1120,758,2.0,NO,NO,YES,NONE,Average,7 Average,Electricity,PUBLIC,1120,550,550,30,2012,0,"2193 Northwest Talus Drive, Issaquah, Washingt...",47.532470,-122.07188


#### Remove Duplicates

In [3]:
#Look for any duplicates in our dataset
kc_family_house_df.duplicated().value_counts()

False    30154
True         1
dtype: int64

Looks like there is a single duplicate to address.

In [4]:
# Dropping the duplicate row
kc_family_house_df.drop_duplicates(keep='first', inplace =True)

In [5]:
#Shape of dataframe (rows, columns)
kc_family_house_df.shape

(30154, 25)

#### Dealing with Missing Values

In [6]:
#View any null values in our data
kc_family_house_df.isna().sum()

id                0
date              0
price             0
bedrooms          0
bathrooms         0
sqft_living       0
sqft_lot          0
floors            0
waterfront        0
greenbelt         0
nuisance          0
view              0
condition         0
grade             0
heat_source      32
sewer_system     14
sqft_above        0
sqft_basement     0
sqft_garage       0
sqft_patio        0
yr_built          0
yr_renovated      0
address           0
lat               0
long              0
dtype: int64

In [7]:
# Dropping any rows with null values since only 46 values out of 30,000+ rows are missing values
kc_family_house_df.dropna(inplace=True)

In [8]:
#Shape of dataframe after dropping records with missing values and columns we don't need
kc_family_house_df.shape

(30110, 25)

Looks like we only dropped 44 rows 

#### Adding Features

Let's split the address into a new column, zip code.

In [9]:
# Example of an address
kc_family_house_df['address'][0]

'2102 Southeast 21st Court, Renton, Washington 98055, United States'

In [10]:
#Separate the Zip Code from the address
kc_family_house_df["Zip Code"] = kc_family_house_df['address'].apply(lambda x: x.split()[-3][:-1])

In [11]:
#Preview our new column
kc_family_house_df["Zip Code"].head()

0    98055
1    98133
2    98178
3    98118
4    98027
Name: Zip Code, dtype: object

#### Remove Outliers

The `column_names.md` file that accompanied the House Sales dataset stated "In some cases due to missing or incorrectly-entered data from the King County Assessor, this API returned locations outside of King County, WA."
<br>
<br>
This means we need to remove houses from our dataset that are not in King County, WA. We pulled zip code information for the county from https://statisticalatlas.com/county/Washington/King-County/Overview.

Compiled King County zip codes into a list to use to subset data frame for only houses that have a zip code in `King_County_WA_zipcodes`.

In [12]:
#View record count before removing Non-King County houses.
kc_family_house_df.shape

(30110, 26)

In [13]:
King_County_WA_zipcodes = ['98001', '98002', '98003', '98004', '98005', '98006', '98007', '98008', '98010', \
    '98011', '98014', '98019', '98021', '98022', '98023', '98024', '98027', '98028', \
    '98029', '98030', '98031', '98032', '98033', '98034', '98038', '98039', '98040', \
    '98042', '98043', '98045', '98047', '98050', '98051', '98052', '98053', '98055', \
    '98056', '98057', '98058', '98059', '98065', '98068', '98070', '98072', '98074', \
    '98075', '98077', '98092', '98101', '98102', '98103', '98104', '98105', '98106', \
    '98107', '98108', '98109', '98112', '98115', '98116', '98117', '98118', '98119', \
    '98121', '98122', '98125', '98126', '98133', '98134', '98136', '98144', '98146', \
    '98148', '98154', '98155', '98158', '98164', '98166', '98168', '98174', '98177', \
    '98178', '98188', '98195', '98198', '98199', '98224', '98288', '98323', '98354', '98391']

In [14]:
kc_family_house_df = kc_family_house_df[kc_family_house_df["Zip Code"].isin(King_County_WA_zipcodes)]

In [15]:
kc_family_house_df.shape

(29171, 26)

We removed 69 house records that were not in King County, WA

### School Districts Dataset

In [16]:
#Import School Districts dataset
schools_df = \
pd.read_csv('/Users/Aidan/Documents/Flatiron/Phase_2/King-County-House-Sales-/data/Report_Card_Assessment_Data_2021-22_School_Year.csv')

/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
#Create a new subset of just the King County Schools
king_school_df = schools_df[schools_df.County == "King"].copy()

In [18]:
#Let's look at our school districts
king_school_df.DistrictName.value_counts()

Seattle School District No. 1              27136
Lake Washington School District            14470
Kent School District                       12693
Federal Way School District                12013
Highline School District                    9768
Northshore School District                  9572
Renton School District                      9106
Bellevue School District                    8219
Auburn School District                      8216
Issaquah School District                    7862
Shoreline School District                   5097
Snoqualmie Valley School District           4148
Tahoma School District                      3478
Enumclaw School District                    3264
Tukwila School District                     2342
Riverview School District                   2162
Mercer Island School District               1849
Vashon Island School District               1528
Rainier Prep Charter School District         698
Summit Public School: Atlas                  592
Skykomish School Dis

Technically Fife School District is in both Pierce and King counties. In the `schools_df`, it is listed as being in Pierce County. We are including it so that the King County houses with Zip Codes corresponding to the Fife School District can be included in our overall dataset.

In [19]:
#Create dataframe with King County School Districts (including Fife)
king_school_df = schools_df[(schools_df.County == "King") |\
                            (schools_df.DistrictName == "Fife School District")].copy()

In [20]:
#Let's look at our school districts to see if Fife is included
king_school_df.DistrictName.value_counts()

Seattle School District No. 1              27136
Lake Washington School District            14470
Kent School District                       12693
Federal Way School District                12013
Highline School District                    9768
Northshore School District                  9572
Renton School District                      9106
Bellevue School District                    8219
Auburn School District                      8216
Issaquah School District                    7862
Shoreline School District                   5097
Snoqualmie Valley School District           4148
Tahoma School District                      3478
Enumclaw School District                    3264
Tukwila School District                     2342
Fife School District                        2215
Riverview School District                   2162
Mercer Island School District               1849
Vashon Island School District               1528
Rainier Prep Charter School District         698
Summit Public School

We also want to remove the schools that are not school districts. <br>
<br>
We only care about the overall scores of each school district and not the scores of each demographic. We will subset our dataframe for District Totals for All Students and All Grades. 

In [21]:
king_school_df = king_school_df[(king_school_df.StudentGroup == "All Students") & \
                                (king_school_df.GradeLevel == "All Grades") & \
                                (king_school_df.SchoolName == 'District Total') &\
                               (king_school_df.DistrictName.str.contains("District"))]

In [22]:
#Limited our dataframe for just the columns we care about. 
king_school_df = king_school_df[['DistrictName','TestAdministration','TestSubject','PercentMetStandard']]

In [23]:
#Checking a single school district to see if it has all the scores we want
king_school_df[king_school_df.DistrictName == "Fife School District"]

,DistrictName,TestAdministration,TestSubject,PercentMetStandard
2130,Fife School District,SBAC,Math,30.8%
74381,Fife School District,WCAS,Science,39.0%
219063,Fife School District,AIM,ELA,36.8%
219078,Fife School District,AIM,Math,42.1%
728240,Fife School District,AIM,Science,Suppressed: N<10
728245,Fife School District,SBAC,ELA,46.2%


Looking at this data, we noticed that each district had two scores for each test subject. By researching the Test Administration, we found that the AIM Scores are for kids with cognitive disabilities has far fewer students in their count. We will separate these scores and use the non-AIM scores (SBAC and WCAS) for our overall rating of a school district as it has a higher count of students. 

In [24]:
#SUBSET OF KING COUNTY SCHOOL DISTRICT SCORES FOR KIDS WITH COGNITIVE DISABILITIES (AIM TEST)
cog_dis_king_school_df = king_school_df[king_school_df['TestAdministration'] == 'AIM']

In [25]:
cog_dis_king_school_df.drop(labels='TestAdministration', axis = 1, inplace=True)

/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [26]:
cog_dis_king_school_df.head()

,DistrictName,TestSubject,PercentMetStandard
2050,Federal Way School District,Science,14.3%
3521,Mercer Island School District,Math,Suppressed: N<10
5926,Shoreline School District,ELA,44.2%
5942,Shoreline School District,Math,44.2%
7171,Vashon Island School District,ELA,Suppressed: N<10


In [27]:
#LARGER SUBSET OF KING COUNTY SCHOOL DISTRICT SCORES FOR KIDS WITHOUT COGNITIVE DISABILITIES 
main_king_school_df = king_school_df[king_school_df['TestAdministration'] != 'AIM']

In [28]:
main_king_school_df.drop(labels='TestAdministration', axis = 1, inplace=True)

/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [29]:
main_king_school_df

,DistrictName,TestSubject,PercentMetStandard
2062,Federal Way School District,ELA,35.6%
2130,Fife School District,Math,30.8%
5384,Renton School District,Math,32.5%
69539,Vashon Island School District,ELA,62.3%
69562,Vashon Island School District,Math,51.3%
...,...,...,...
730736,Enumclaw School District,Science,49.8%
735027,Bellevue School District,Math,65.6%
735906,Auburn School District,ELA,43.6%
735909,Auburn School District,Math,30.2%


#### Pivot Dataset

We want to turn our dataset into a dataframe with `DistrictName` as rows and `TestSubject` as Columns with the corresponding `PercentMetStandard` at their intersection. 

In [30]:
#Create a pivot table
main_king_school_df = main_king_school_df.pivot(index = "DistrictName", columns = "TestSubject",\
                                                values = "PercentMetStandard")

In [31]:
#Reset the index
main_king_school_df= main_king_school_df.reset_index()

In [32]:
#View the columns
main_king_school_df.columns

Index(['DistrictName', 'ELA', 'Math', 'Science'], dtype='object', name='TestSubject')

In [33]:
#View our updated pivot table
main_king_school_df

TestSubject,DistrictName,ELA,Math,Science
0,Auburn School District,43.6%,30.2%,35.1%
1,Bellevue School District,72.3%,65.6%,63.7%
2,Enumclaw School District,56.7%,40.1%,49.8%
3,Federal Way School District,35.6%,21.7%,27.1%
4,Fife School District,46.2%,30.8%,39.0%
5,Highline School District,33.4%,21.5%,26.8%
6,Issaquah School District,76.0%,69.6%,63.2%
7,Kent School District,47.9%,33.7%,39.7%
8,Lake Washington School District,78.5%,70.2%,61.6%
9,Mercer Island School District,82.0%,76.9%,75.9%


It looks like one of the districts is `Rainier Prep Charter School District` which is a charter school and so does not reflect the school districts available based on house location. We will drop this row.

In [34]:
main_king_school_df = main_king_school_df[main_king_school_df['DistrictName']\
                                          != "Rainier Prep Charter School District"]

In [35]:
#Resetrting the index name
main_king_school_df.index.name = 'index'

In [36]:
# Confirming Rainier Prep Charter School District is no longer in our dataframe.
main_king_school_df

TestSubject,DistrictName,ELA,Math,Science
index,,,,
0,Auburn School District,43.6%,30.2%,35.1%
1,Bellevue School District,72.3%,65.6%,63.7%
2,Enumclaw School District,56.7%,40.1%,49.8%
3,Federal Way School District,35.6%,21.7%,27.1%
4,Fife School District,46.2%,30.8%,39.0%
5,Highline School District,33.4%,21.5%,26.8%
6,Issaquah School District,76.0%,69.6%,63.2%
7,Kent School District,47.9%,33.7%,39.7%
8,Lake Washington School District,78.5%,70.2%,61.6%


In [37]:
#Resetting index so we have 0 to 19
main_king_school_df.reset_index(drop = True, inplace=True)

In [38]:
#Renaming to remove "TestSubject"
main_king_school_df.rename_axis(None, axis =1, inplace=True)

### Add New Feature- Comparison Score for School Districts

For comparison purposes, let's create another column with the overall district test score called `District Test Score`

In [39]:
#First we need to strip the "%" sign from the Test Subject scores
main_king_school_df = main_king_school_df.apply(lambda x: x.str.strip('%'))

In [40]:
#Replace Skykomish School District Science value "Suppressed: N<10" with "0" as a placeholder
main_king_school_df.replace(to_replace='Suppressed: N<10', value= 'NaN', inplace=True)

In [41]:
#Turn our ELA, Math, and Science values into floats
main_king_school_df[['ELA','Math','Science']] = main_king_school_df[['ELA','Math','Science']].astype(float)

In [42]:
#Create For Loop for adding `District Test Score` column to corresponding Districts
for index, district in enumerate(main_king_school_df['DistrictName']):
    main_king_school_df.loc[index,'District Test Score'] = round((main_king_school_df['ELA'][index] +\
    main_king_school_df['Math'][index] + main_king_school_df['Science'][index])/3, 2)

In [43]:
#Look at our dataframe to ensure our District Test Score column was created.
main_king_school_df

,DistrictName,ELA,Math,Science,District Test Score
0,Auburn School District,43.6,30.2,35.1,36.30
1,Bellevue School District,72.3,65.6,63.7,67.20
2,Enumclaw School District,56.7,40.1,49.8,48.87
3,Federal Way School District,35.6,21.7,27.1,28.13
4,Fife School District,46.2,30.8,39.0,38.67
5,Highline School District,33.4,21.5,26.8,27.23
6,Issaquah School District,76.0,69.6,63.2,69.60
7,Kent School District,47.9,33.7,39.7,40.43
8,Lake Washington School District,78.5,70.2,61.6,70.10
9,Mercer Island School District,82.0,76.9,75.9,78.27


We still need to replace Skykomish School District's `District Test Score` using the average of the only two scores we have for `ELA` and `Math`

In [44]:
main_king_school_df['District Test Score'][15] = \
(main_king_school_df['ELA'][15] + main_king_school_df['Math'][15])/2

<ipython-input-44-a2a8ec8a7121>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_king_school_df['District Test Score'][15] = \


In [45]:
#Checking that Skykomish School District has an updated District Test Score
main_king_school_df

,DistrictName,ELA,Math,Science,District Test Score
0,Auburn School District,43.6,30.2,35.1,36.30
1,Bellevue School District,72.3,65.6,63.7,67.20
2,Enumclaw School District,56.7,40.1,49.8,48.87
3,Federal Way School District,35.6,21.7,27.1,28.13
4,Fife School District,46.2,30.8,39.0,38.67
5,Highline School District,33.4,21.5,26.8,27.23
6,Issaquah School District,76.0,69.6,63.2,69.60
7,Kent School District,47.9,33.7,39.7,40.43
8,Lake Washington School District,78.5,70.2,61.6,70.10
9,Mercer Island School District,82.0,76.9,75.9,78.27


### Add New Feature- School District Zip Codes

Create dictionary `King_County_School_District_Zipcodes` with zipcodes for each school district in King County, WA. Information taken from https://statisticalatlas.com/county/Washington/King-County/Overview

In [46]:
King_County_School_District_Zipcodes = {

'Seattle School District No. 1' : ['98101', '98102', '98103', '98104', '98105', '98106', '98107', '98108',\
                                    '98109', '98112', '98115', '98116', '98117', '98118', '98119', '98121',\
                                    '98122', '98125', '98126', '98133', '98134', '98136', '98144', '98146',\
                                    '98154', '98164', '98168', '98174', '98177', '98178', '98195', '98199'],

'Lake Washington School District' : ['98004', '98005', '98007', '98008', '98011', '98033', '98034',\
                                            '98039', '98052', '98053', '98072', '98074', '98075', '98077'],

'Kent School District' : ['98001', '98002', '98010', '98030', '98031', '98032', '98038', '98042', '98055',\
                                 '98058', '98092', '98188', '98198'],

'Federal Way School District' : ['98001', '98003', '98023', '98032', '98198', '98354'],

'Highline School District': ['98032', '98106', '98108', '98126', '98146', '98148', '98158', '98166',\
                                     '98168', '98188', '98198'],

'Northshore School District' : ['98011', '98012', '98021', '98028', '98034', '98036', '98052', '98072',\
                                       '98077', '98155', '98296'],

'Renton School District' : ['98006', '98031', '98032', '98055', '98056', '98057', '98058', '98059',\
                                   '98168', '98178', '98188'],

'Bellevue School District' : ['98004', '98005', '98006', '98007', '98008', '98027', '98033', '98039',\
                                     '98052', '98056', '98059'],

'Auburn School District' : ['98001', '98002', '98010', '98022', '98030', '98032', '98042', '98047',\
                                   '98092', '98391'],

'Issaquah School District': ['98006', '98008', '98024', '98027', '98029', '98038', '98050', '98056',\
                                     '98058', '98059', '98065', '98074', '98075'],

'Shoreline School District' : ['98043', '98133', '98155', '98177'],

'Snoqualmie Valley School District' : ['98014', '98024', '98027', '98038', '98045', '98053', '98065',\
                                              '98068', '98074', '98075', '98224'],

'Tahoma School District' : ['98010', '98027', '98038', '98042', '98045', '98051', '98058',\
                                   '98059', '98065'],

'Enumclaw School District': ['98010', '98022', '98038', '98045', '98051', '98092', '98323', '98391'],

'Tukwila School District' : ['98168', '98178', '98188'],

'Riverview School District' : ['98014', '98019', '98024', '98053', '98065', '98077', '98224'],

'Mercer Island School District' : ['98004', '98006', '98040'],

'Vashon Island School District' : ['98070'],

'Skykomish School District' : ['98045', '98068', '98224', '98288'],

'Fife School District' : ['98001', '98003', '98354', '98371', '98372', '98421', '98422', '98424']
    
}    

In [47]:
#Create For Loop for adding Zip Code column to corresponding School District
for index,district in enumerate(main_king_school_df.DistrictName):
    if district in King_County_School_District_Zipcodes.keys():
        main_king_school_df.loc[index,'zipcodes'] = (",".join(King_County_School_District_Zipcodes[district]))
        

In [48]:
#Create new column `zipcodes` which contains all the zipcodes corresponding to each school district
main_king_school_df['zipcodes'] = main_king_school_df['zipcodes'].apply(lambda x: str(x).split(','))

In [49]:
#View our dataframe with our new `zipcodes` column
main_king_school_df

,DistrictName,ELA,Math,Science,District Test Score,zipcodes
0,Auburn School District,43.6,30.2,35.1,36.30,"[98001, 98002, 98010, 98022, 98030, 98032, 980..."
1,Bellevue School District,72.3,65.6,63.7,67.20,"[98004, 98005, 98006, 98007, 98008, 98027, 980..."
2,Enumclaw School District,56.7,40.1,49.8,48.87,"[98010, 98022, 98038, 98045, 98051, 98092, 983..."
3,Federal Way School District,35.6,21.7,27.1,28.13,"[98001, 98003, 98023, 98032, 98198, 98354]"
4,Fife School District,46.2,30.8,39.0,38.67,"[98001, 98003, 98354, 98371, 98372, 98421, 984..."
5,Highline School District,33.4,21.5,26.8,27.23,"[98032, 98106, 98108, 98126, 98146, 98148, 981..."
6,Issaquah School District,76.0,69.6,63.2,69.60,"[98006, 98008, 98024, 98027, 98029, 98038, 980..."
7,Kent School District,47.9,33.7,39.7,40.43,"[98001, 98002, 98010, 98030, 98031, 98032, 980..."
8,Lake Washington School District,78.5,70.2,61.6,70.10,"[98004, 98005, 98007, 98008, 98011, 98033, 980..."
9,Mercer Island School District,82.0,76.9,75.9,78.27,"[98004, 98006, 98040]"


 ## Adding District Name to House Sales Dataset

In [50]:
#Resetting the index
kc_family_house_df.reset_index(inplace = True)

In [51]:
#Create For Loop for adding DistrictName column to corresponding Houses based on Zip Code
for index, zipcode in enumerate(kc_family_house_df['Zip Code']):
    
    for district in King_County_School_District_Zipcodes.keys():
     
        if zipcode in King_County_School_District_Zipcodes[district]:
            
            kc_family_house_df.loc[index ,'DistrictName']= district

In [52]:
#Checking the shape of our resulting dataframe
kc_family_house_df.shape

(29171, 28)

In [53]:
#Previewing our dataframe to see our new DistrictName column
kc_family_house_df.head()

,index,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,Zip Code,DistrictName
0,0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,NO,NONE,Good,7 Average,Gas,PUBLIC,1180,0,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.19052,98055,Renton School District
1,1,8910500230,12/13/2021,920000.0,5,2.5,2770,6703,1.0,NO,NO,YES,AVERAGE,Average,7 Average,Oil,PUBLIC,1570,1570,0,240,1950,0,"11231 Greenwood Avenue North, Seattle, Washing...",47.711525,-122.35591,98133,Shoreline School District
2,2,1180000275,9/29/2021,311000.0,6,2.0,2880,6156,1.0,NO,NO,NO,AVERAGE,Average,7 Average,Gas,PUBLIC,1580,1580,0,0,1956,0,"8504 South 113th Street, Seattle, Washington 9...",47.502045,-122.22520,98178,Tukwila School District
3,3,1604601802,12/14/2021,775000.0,3,3.0,2160,1400,2.0,NO,NO,NO,AVERAGE,Average,9 Better,Gas,PUBLIC,1090,1070,200,270,2010,0,"4079 Letitia Avenue South, Seattle, Washington...",47.566110,-122.29020,98118,Seattle School District No. 1
4,4,8562780790,8/24/2021,592500.0,2,2.0,1120,758,2.0,NO,NO,YES,NONE,Average,7 Average,Electricity,PUBLIC,1120,550,550,30,2012,0,"2193 Northwest Talus Drive, Issaquah, Washingt...",47.532470,-122.07188,98027,Tahoma School District


 ## Consolidating Dataframes

Joining our school districts dataframe onto the housing data set by `DistrictName` so that every house has a column with it's appropriate school district.

In [54]:
kc_house_school_df = \
kc_family_house_df.merge(main_king_school_df, left_on = 'DistrictName', right_on ='DistrictName')

In [55]:
kc_house_school_df.head()

,index,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,Zip Code,DistrictName,ELA,Math,Science,District Test Score,zipcodes
0,0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,NO,NONE,Good,7 Average,Gas,PUBLIC,1180,0,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.19052,98055,Renton School District,42.9,32.5,33.3,36.23,"[98006, 98031, 98032, 98055, 98056, 98057, 980..."
1,11,2873000690,6/11/2021,680000.0,4,3.0,2130,7649,1.0,NO,NO,NO,NONE,Very Good,7 Average,Electricity,PUBLIC,1130,1100,440,280,1975,0,"20432 130th Place Southeast, Kent, Washington ...",47.418155,-122.16696,98031,Renton School District,42.9,32.5,33.3,36.23,"[98006, 98031, 98032, 98055, 98056, 98057, 980..."
2,22,7231501575,12/28/2021,450000.0,2,1.0,900,6000,1.0,NO,NO,NO,NONE,Average,6 Low Average,Gas,PUBLIC,810,810,0,0,1941,0,"533 Wells Avenue South, Renton, Washington 980...",47.475360,-122.20573,98057,Renton School District,42.9,32.5,33.3,36.23,"[98006, 98031, 98032, 98055, 98056, 98057, 980..."
3,25,2095600170,10/8/2021,580000.0,3,3.0,2020,4482,1.0,NO,NO,NO,NONE,Average,7 Average,Gas,PUBLIC,1370,1250,600,120,1992,0,"22609 102nd Place Southeast, Kent, Washington ...",47.399470,-122.20413,98031,Renton School District,42.9,32.5,33.3,36.23,"[98006, 98031, 98032, 98055, 98056, 98057, 980..."
4,30,9320600110,8/18/2021,500000.0,3,1.5,1140,9690,1.0,NO,NO,NO,NONE,Good,7 Average,Gas,PUBLIC,1140,0,550,0,1964,0,"9812 South 212th Street, Kent, Washington 9803...",47.411835,-122.20954,98031,Renton School District,42.9,32.5,33.3,36.23,"[98006, 98031, 98032, 98055, 98056, 98057, 980..."


In [56]:
#Checking the shape of our resulting dataframe- we added 5 columns
kc_house_school_df.shape

(29171, 33)

In [57]:
# Checking if there are any null (NaN) values.
kc_house_school_df.isna().sum()

index                    0
id                       0
date                     0
price                    0
bedrooms                 0
bathrooms                0
sqft_living              0
sqft_lot                 0
floors                   0
waterfront               0
greenbelt                0
nuisance                 0
view                     0
condition                0
grade                    0
heat_source              0
sewer_system             0
sqft_above               0
sqft_basement            0
sqft_garage              0
sqft_patio               0
yr_built                 0
yr_renovated             0
address                  0
lat                      0
long                     0
Zip Code                 0
DistrictName             0
ELA                      0
Math                     0
Science                485
District Test Score      0
zipcodes                 0
dtype: int64

It looks like the only place we are missing values are in the `Science` column, and we know that is for the Skykomish School District.

Let's export our final dataset as a csv for us to use for modeling. 

In [58]:
final_kc_csv = kc_house_school_df.to_csv('final_kc.csv')